In [1]:
import numpy as np
import pandas as pd
import os
from multiprocessing import Pool
from tqdm import tqdm

from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore') 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

pd.set_option('expand_frame_repr', True) 
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)
pd.set_option('display.width', 180) 

In [2]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, Dataset

In [3]:
from data_api import DataFeeder, DataAPI
from factor_base import FactorBase

# Instantiate the data interface
d = DataFeeder() 
api = DataAPI()

函数: read_stk_data 运行了 0.0082秒
函数: read_stk_data 运行了 2.8756秒
函数: read_stk_data 运行了 0.0043秒
函数: read_stk_data 运行了 2.8428秒


In [4]:
data = pd.read_feather("/mnt/data/stocks/basic/processed_stock_trade_data.feather")
data = data[data.date >= "2016-01-01"][['date', 'stock_code', 'open', 'high', 'low', 'close', 'volume', 'total_turnover']]
data['vwap'] = data.total_turnover / data.volume
date_list = data.date.unique().tolist()
def process_stock_group(group):
    group = group[['date', 'vwap']].copy()
    group.set_index('date', inplace=True)
    group = group.reindex(date_list)
    group["return_5"] = (group.vwap.shift(-4) / group.vwap - 1).shift(-1)
    return group.iloc[:-5,:]
df_return = data.groupby('stock_code').apply(process_stock_group).reset_index()
df_return.rename(columns={'level_1': 'date'}, inplace=True)
df_return = df_return[['date', 'stock_code', 'return_5']]
df_return.to_feather("/mnt/research/data/temp/zhangsurui/E2E_NN/Round2/return_data.feather")
df_return


,date,stock_code,return_5
0,2016-01-04,000001.SZ,-0.039761
1,2016-01-05,000001.SZ,-0.059640
2,2016-01-06,000001.SZ,-0.028410
3,2016-01-07,000001.SZ,-0.043834
4,2016-01-08,000001.SZ,-0.031289
...,...,...,...
10053977,2023-11-22,689009.SH,-0.023394
10053978,2023-11-23,689009.SH,-0.069582
10053979,2023-11-24,689009.SH,-0.045808
10053980,2023-11-27,689009.SH,-0.017599


In [5]:
def process_stock(stock_seq):
    stock = stack_index[stock_seq]
    if stock in df_rtn.index:
        stock_rtn = df_rtn.loc[stock, "return_5"]
        if ~np.isnan(stock_rtn):
            y = stock_rtn
            return y, stock_seq, stock
    return None, None, None

n_jobs = 150
window_size = 5
train_sample_list = []
train_index_list = []
test_sample_list = []
test_index_list = []

for date_seq in tqdm(range(window_size-1,window_size+1)):#len(date_list))):
    
    date_current = date_list[date_seq]
    df_rtn = df_return.loc[df_return.date == date_current, ["stock_code", "return_5"]].set_index("stock_code")
    data_type = "train" if date_current.strftime("%Y-%m-%d") < "2022-01-01" else "test"

    date_window = date_list[date_seq-window_size+1:date_seq+1]
    window_stack = []
    for date in date_window:
        data_mink = pd.read_feather("/mnt/data/stocks/mink/by_date/"+date.strftime("%Y%m%d")+".feather")
        data_mink["stock_code"] = data_mink.order_book_id.map(lambda x:api.code_transf(rqcode=x,verse=False))
        data_mink["vwap"] = data_mink.total_turnover / data_mink.volume
        data_mink = data_mink[["stock_code", "datetime", "date", "time", "open", "high", "low", "close", "vwap", "volume"]]
        data_mink = data_mink[(data_mink.time != "14:58:00") & (data_mink.time != "14:59:00")]
        data_mink.loc[data_mink.volume == 0, "vwap"] = data_mink.loc[data_mink.volume == 0, "open"]
        window_stack.append(data_mink)
    df_window = pd.concat(window_stack, axis=0)
    if df_window.isna().any().any():
        print("The window " + date_current.strftime("%Y%m%d") + " contains unexpected NaN")
    stack_index = []
    stack_numpy = []
    for df in df_window.groupby("stock_code"):
        if len(df[1]) == 1190:
            stack_index.append(df[0])
            stack_numpy.append(df[1][["open", "high", "low", "close", "vwap", "volume"]].to_numpy())
    stack_numpy = np.array(stack_numpy)
    # ts_pre_process
    price_adj = np.tile(stack_numpy[:,0,0][:,np.newaxis], (1,stack_numpy.shape[1]))
    for price in range(5):
        stack_numpy[:,:,price] /= price_adj
    volume_adj = stack_numpy[:,:,5].mean(axis=1)
    volume_adj = np.tile(np.where(volume_adj < 1e-14, 1, volume_adj)[:,np.newaxis], (1,stack_numpy.shape[1]))
    stack_numpy[:,:,5] /= volume_adj
    # cs_pre_process
    for feat in range(6):
        cs_mean = np.tile(stack_numpy[:,:,feat].std(axis=0)[np.newaxis,:], (stack_numpy.shape[0],1))
        cs_std = np.tile(stack_numpy[:,:,feat].std(axis=0)[np.newaxis,:], (stack_numpy.shape[0],1))
        stack_numpy[:,:,feat] = np.where(cs_std < 1e-14, np.zeros_like(stack_numpy[:,:,feat]), (stack_numpy[:,:,feat] - cs_mean) / cs_std)

    # # df_window = df_window.groupby("stock_code").apply(ts_pre_process).reset_index(drop=True)
    # # df_window = df_window.groupby("datetime").apply(cs_pre_process).reset_index(drop=True)
    # # df_window.to_feather("/mnt/research/data/temp/zhangsurui/E2E_NN/Round2/"+data_type+"/"+date_current.strftime("%Y%m%d")+"_"+data_type+".feather")

    pool = Pool(n_jobs)
    results = pool.map(process_stock, range(len(stack_index)))
    pool.close()
    pool.join()
    batch_list = []
    batch_y = []
    batch_code = []
    for result in results:
        y, index, code = result
        if y is not None:
            batch_y.append(y)
            batch_list.append(index)
            batch_code.append(code)
    batch_X = torch.Tensor(stack_numpy[batch_list])

    if data_type == "train":
        train_sample_list.append((batch_X, batch_y))
        train_index_list.append(batch_code)
    else:
        test_sample_list.append((batch_X, batch_y))
        test_index_list.append(batch_code)

100%|██████████| 2/2 [01:58<00:00, 59.10s/it]


In [23]:
def ts_pre_process(group):
    price_adj = group.open.tolist()[0]
    group.open = group.open / price_adj
    group.high = group.high / price_adj
    group.low = group.low / price_adj
    group.close = group.close / price_adj
    group.vwap = group.vwap / price_adj

    volume_adj = group.volume.mean()
    if volume_adj < 1e-14:
        volume_adj = 1
    group.volume = group.volume / volume_adj

    return group

def cs_pre_process(group):
    if group.open.std() < 1e-14:
        group.open = 0
    else:
        group.open = (group.open - group.open.mean()) / group.open.std()
    group.high = (group.high - group.high.mean()) / group.high.std()
    group.low = (group.low - group.low.mean()) / group.low.std()
    group.close = (group.close - group.close.mean()) / group.close.std()
    group.vwap = (group.vwap - group.vwap.mean()) / group.vwap.std()

    if group.volume.std() < 1e-14:
        group.volume = 0
    else:
        group.volume = (group.volume - group.volume.mean()) / group.volume.std()

    return group

def process_stock(stock_seq):
    stock = stack_index[stock_seq]
    if stock in df_rtn.index:
        stock_rtn = df_rtn.loc[stock, "return_5"]
        if ~np.isnan(stock_rtn):
            y = stock_rtn
            return y, stock_seq, stock
    return None, None, None
    
n_jobs = 150
window_size = 5
train_sample_list = []
train_index_list = []
test_sample_list = []
test_index_list = []

date_seq = window_size-1
date_current = date_list[date_seq]
df_rtn = df_return.loc[df_return.date == date_current, ["stock_code", "return_5"]].set_index("stock_code")
data_type = "train" if date_current.strftime("%Y-%m-%d") < "2022-01-01" else "test"

date_window = date_list[date_seq-window_size+1:date_seq+1]
window_stack = []
for date in date_window:
    data_mink = pd.read_feather("/mnt/data/stocks/mink/by_date/"+date.strftime("%Y%m%d")+".feather")
    data_mink["stock_code"] = data_mink.order_book_id.map(lambda x:api.code_transf(rqcode=x,verse=False))
    data_mink["vwap"] = data_mink.total_turnover / data_mink.volume
    data_mink = data_mink[["stock_code", "datetime", "date", "time", "open", "high", "low", "close", "vwap", "volume"]]
    data_mink = data_mink[(data_mink.time != "14:58:00") & (data_mink.time != "14:59:00")]
    data_mink.loc[data_mink.volume == 0, "vwap"] = data_mink.loc[data_mink.volume == 0, "open"]
    window_stack.append(data_mink)
df_window = pd.concat(window_stack, axis=0)
if df_window.isna().any().any():
    print("The window " + date_current.strftime("%Y%m%d") + " contains unexpected NaN")
stack_index = []
stack_numpy = []
for df in df_window.groupby("stock_code"):
    if len(df[1]) == 1190:
        stack_index.append(df[0])
        stack_numpy.append(df[1][["open", "high", "low", "close", "vwap", "volume"]].to_numpy())
stack_numpy = np.array(stack_numpy)
# ts_pre_process
price_adj = np.tile(stack_numpy[:,0,0][:,np.newaxis], (1,stack_numpy.shape[1]))
for price in range(5):
    stack_numpy[:,:,price] /= price_adj
volume_adj = stack_numpy[:,:,5].mean(axis=1)
volume_adj = np.tile(np.where(volume_adj < 1e-14, 1, volume_adj)[:,np.newaxis], (1,stack_numpy.shape[1]))
stack_numpy[:,:,5] /= volume_adj
# cs_pre_process
for feat in range(6):
    cs_mean = np.tile(stack_numpy[:,:,feat].std(axis=0)[np.newaxis,:], (stack_numpy.shape[0],1))
    cs_std = np.tile(stack_numpy[:,:,feat].std(axis=0)[np.newaxis,:], (stack_numpy.shape[0],1))
    stack_numpy[:,:,feat] = np.where(cs_std < 1e-14, np.zeros_like(stack_numpy[:,:,feat]), (stack_numpy[:,:,feat] - cs_mean) / cs_std)

# # df_window = df_window.groupby("stock_code").apply(ts_pre_process).reset_index(drop=True)
# # df_window = df_window.groupby("datetime").apply(cs_pre_process).reset_index(drop=True)
# # df_window.to_feather("/mnt/research/data/temp/zhangsurui/E2E_NN/Round2/"+data_type+"/"+date_current.strftime("%Y%m%d")+"_"+data_type+".feather")

pool = Pool(n_jobs)
results = pool.map(process_stock, range(len(stack_index)))
pool.close()
pool.join()
batch_list = []
batch_y = []
batch_code = []
for result in results:
    y, index, code = result
    if y is not None:
        batch_y.append(y)
        batch_list.append(index)
        batch_code.append(code)
batch_X = torch.Tensor(stack_numpy[batch_list])

if data_type == "train":
    train_sample_list.append((batch_X, batch_y))
    train_index_list.append(batch_code)
else:
    test_sample_list.append((batch_X, batch_y))
    test_index_list.append(batch_code)

In [78]:
stack_numpy[[0,2]].shape

(2, 1190, 6)

In [72]:
stack_index = []
stack_numpy = []
for df in df_window.groupby("stock_code"):
    if len(df[1]) == 1190:
        stack_index.append(df[0])
        stack_numpy.append(df[1][["open", "high", "low", "close", "vwap", "volume"]].to_numpy())
stack_numpy = np.array(stack_numpy)

In [73]:
price_adj = np.tile(stack_numpy[:,0,0][:,np.newaxis], (1,stack_numpy.shape[1]))
for price in range(5):
    stack_numpy[:,:,price] /= price_adj
volume_adj = stack_numpy[:,:,5].mean(axis=1)
volume_adj = np.tile(np.where(volume_adj < 1e-14, 1, volume_adj)[:,np.newaxis], (1,stack_numpy.shape[1]))
stack_numpy[:,:,5] /= volume_adj
stack_numpy

array([[[ 1.        ,  1.0025    ,  0.99916667,  0.99916667,
          1.00021267,  3.03686472],
        [ 0.99916667,  0.99916667,  0.99666667,  0.9975    ,
          0.99864011,  2.95327965],
        [ 0.9975    ,  0.99916667,  0.99666667,  0.99916667,
          0.99797085,  1.64204107],
        ...,
        [ 0.92666667,  0.92833333,  0.92666667,  0.9275    ,
          0.92724031,  0.82995438],
        [ 0.92833333,  0.92833333,  0.92666667,  0.92666667,
          0.92752923,  1.51696274],
        [ 0.9275    ,  0.9275    ,  0.92666667,  0.92666667,
          0.92666671,  1.90666161]],

       [[ 1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  0.        ],
        [ 1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  0.        ],
        [ 1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  0.        ],
        ...,
        [ 1.        ,  1.        ,  1.        ,  1.        ,
          1.        ,  0.        ],
  

In [74]:
for feat in range(6):
    cs_mean = np.tile(stack_numpy[:,:,feat].std(axis=0)[np.newaxis,:], (stack_numpy.shape[0],1))
    cs_std = np.tile(stack_numpy[:,:,feat].std(axis=0)[np.newaxis,:], (stack_numpy.shape[0],1))
    stack_numpy[:,:,feat] = np.where(cs_std < 1e-14, np.zeros_like(stack_numpy[:,:,feat]), (stack_numpy[:,:,feat] - cs_mean) / cs_std)
stack_numpy

array([[[ 0.00000000e+00,  1.74732450e+02,  1.62073142e+02,
          1.42466780e+02,  3.10300316e+02, -4.94967366e-01],
        [ 1.30320334e+02,  1.24022201e+02,  1.18176344e+02,
          1.19637115e+02,  1.28261577e+02,  1.63917026e-01],
        [ 1.14272741e+02,  1.10745924e+02,  1.06381424e+02,
          1.06608291e+02,  1.11144693e+02,  3.27889133e-02],
        ...,
        [ 7.97780243e+00,  8.00292446e+00,  7.96181674e+00,
          7.97817043e+00,  7.97944862e+00, -3.94487419e-01],
        [ 7.98676947e+00,  7.99776655e+00,  7.94878509e+00,
          7.96413150e+00,  7.97286339e+00,  6.44946750e-02],
        [ 7.97291173e+00,  7.98069770e+00,  7.95276898e+00,
          7.96157022e+00,  7.96171829e+00, -2.63287447e-01]],

       [[ 0.00000000e+00,  1.74294215e+02,  1.62209149e+02,
          1.42586436e+02,  3.10234127e+02, -1.00000000e+00],
        [ 1.30429859e+02,  1.24126473e+02,  1.18574928e+02,
          1.19939463e+02,  1.28437597e+02, -1.00000000e+00],
        [ 1.14561

In [30]:
data_mink_0 = pd.read_feather("/mnt/data/stocks/mink/by_date/20231204.feather")
data_mink_0["stock_code"] = data_mink_0.order_book_id.map(lambda x:api.code_transf(rqcode=x,verse=False))
data_mink_0["vwap"] = data_mink_0.total_turnover / data_mink_0.volume
data_mink_0 = data_mink_0[["stock_code", "datetime", "date", "time", "open", "high", "low", "close", "vwap", "volume"]]
data_mink_0 = data_mink_0[(data_mink_0.time != "14:58:00") & (data_mink_0.time != "14:59:00")]
data_mink_0.loc[data_mink_0.volume == 0, "vwap"] = data_mink_0.loc[data_mink_0.volume == 0, "open"]
data_mink_0

,stock_code,datetime,date,time,open,high,low,close,vwap,volume
0,000001.SZ,2023-12-04 09:31:00,2023-12-04,09:31:00,9.67,9.67,9.65,9.66,9.663389,1926400.0
1,000001.SZ,2023-12-04 09:32:00,2023-12-04,09:32:00,9.66,9.68,9.65,9.65,9.665134,1959100.0
2,000001.SZ,2023-12-04 09:33:00,2023-12-04,09:33:00,9.66,9.66,9.65,9.65,9.653076,649000.0
3,000001.SZ,2023-12-04 09:34:00,2023-12-04,09:34:00,9.65,9.65,9.64,9.65,9.647179,532700.0
4,000001.SZ,2023-12-04 09:35:00,2023-12-04,09:35:00,9.64,9.65,9.63,9.64,9.636993,669800.0
...,...,...,...,...,...,...,...,...,...,...
1219673,689009.SH,2023-12-04 14:54:00,2023-12-04,14:54:00,33.61,33.62,33.60,33.60,33.607624,11517.0
1219674,689009.SH,2023-12-04 14:55:00,2023-12-04,14:55:00,33.61,33.62,33.60,33.61,33.609957,7412.0
1219675,689009.SH,2023-12-04 14:56:00,2023-12-04,14:56:00,33.58,33.62,33.57,33.57,33.587015,17359.0
1219676,689009.SH,2023-12-04 14:57:00,2023-12-04,14:57:00,33.57,33.58,33.55,33.57,33.565413,24911.0


In [18]:
np.array(train_sample_list[0][0][0].squeeze()).shape

(1190, 6)

In [19]:
X = torch.Tensor([np.array(x.squeeze()) for x in train_sample_list[0][0]])
X.shape


ValueError: expected sequence of length 1190 at dim 1 (got 714)

In [12]:
x = train_sample_list[0][0][0]
x.shape
x.size(0)

torch.Size([1, 1190, 6])

1

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class PearsonCorrelationLoss(nn.Module):
    def forward(self, y_pred, y_true):
        y_pred_mean = torch.mean(y_pred)
        y_true_mean = torch.mean(y_true)
        y_pred_std = torch.std(y_pred)
        y_true_std = torch.std(y_true)

        cov = torch.mean((y_pred - y_pred_mean) * (y_true - y_true_mean))
        IC = cov / (y_pred_std * y_true_std)

        return -IC

class GRUNet(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=1, output_size=1, dropout_rate=0.4,random_seed=0):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_rate if num_layers > 1 else 0)
        # self.batch_norm = nn.BatchNorm1d(hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        # out = self.batch_norm(out)
        out = self.fc(out[:, -1, :])
        return out

model = GRUNet(input_size=6, hidden_size=50, num_layers=2, output_size=1, dropout_rate=0.4)
criterion = PearsonCorrelationLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

num_epochs = 3
model.to(device)
train_losses = []
train_acc = []
test_losses = []
test_acc = []

for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    train_correct = 0
    total_train = 0

    for seq, labels in train_sample_list:
        seq, labels = seq.to(device), labels.to(device)
        optimizer.zero_grad()
        for X_individual in seq:
            y_individual = model(X_individual.float())
            
        y_pred = model(seq.float())
        single_loss = criterion(y_pred.squeeze(), labels.float().squeeze())
        single_loss.backward()
        optimizer.step()
        
        total_train_loss += single_loss.item()

        

    
    model.eval()
    total_test_loss = 0
    test_correct = 0
    total_test = 0

    for seq, labels in test_loader:
        seq, labels = seq.to(device), labels.to(device)
        y_pred = model(seq.float())
        single_loss = criterion(y_pred.squeeze(), labels.float().squeeze())
        
        total_test_loss += single_loss.item()
        predicted = (y_pred > 0.5).float()
        total_test += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        
    test_loss = total_test_loss / len(test_loader)
    test_accuracy = test_correct / total_test
    test_losses.append(test_loss)
    test_acc.append(test_accuracy)

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}')

torch.save(model.state_dict(), '/mnt/research/data/temp/zhangsurui/E2E_NN/Round1/model_state_dict.pth')
print('Finished Training')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a custom Pearson correlation loss
class PearsonCorrelationLoss(nn.Module):
    def forward(self, y_pred, y_true):
        y_pred_mean = torch.mean(y_pred)
        y_true_mean = torch.mean(y_true)
        y_pred_std = torch.std(y_pred)
        y_true_std = torch.std(y_true)

        covariance = torch.mean((y_pred - y_pred_mean) * (y_true - y_true_mean))
        correlation = covariance / (y_pred_std * y_true_std)

        # Return the negative of Pearson correlation for minimization
        return -correlation

# Example GRU model
class GRUNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUNet, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        output, _ = self.gru(x)
        output = self.linear(output[-1])
        return output

# Initialize model, loss, and optimizer
model = GRUNet(input_size, hidden_size, output_size)
criterion = PearsonCorrelationLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Example training loop
for epoch in range(epochs):
    for x_batch, y_batch in train_loader:
        # Forward pass
        y_pred = model(x_batch)
        loss = criterion(y_pred, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
